In [19]:
import importlib
rs = importlib.reload(rs)

In [2]:
import numpy as np
import pandas as pd
import recsys as rs
import json
import notipy
import random
import TopSimilarRecommender as TSR
import ItemBasedRecommender as IBR

In [3]:
train = pd.read_csv('Data/train_final.csv','\t')
tr_info = pd.read_csv('Data/tracks_final.csv','\t')

In [4]:
train, test, tgt_tracks, tgt_playlists = rs.split_train_test(train, 10, 20, 5, 2517)

In [5]:
fit_dict_tsr = {'attributes' : ['artist_id', 'album', 'tags', 'playcount'],
                'attributes_to_prune' : ['tags'],
                'n_min_attr' : 90,
                'idf' : True,
                'measure' : 'dot',
                'shrinkage' : 0,
                'n_el_sim' : 15}

fit_dict_ibr = {'idf' : True,
                'measure' : 'dot',
                'shrinkage' : 0,
                'n_el_sim' : 15}

In [6]:
tsr = TSR.TopSimilarRecommender(**fit_dict_tsr)
tsr.fit(tr_info, tgt_tracks)
notipy.notify('TSR fitted!')

Fixed dataset
Calculated Indices
Eliminated low frequency attributes!
ICM built
ICM regularized with IDF!


/Users/LucaButera/git/rschallenge/recsys.py:336: RuntimeWarning: divide by zero encountered in true_divide
  return np.log10(n_items / frequencies)
100%|██████████| 100000/100000 [14:47<00:00, 112.69it/s]


Similarity built


In [7]:
ibr = IBR.ItemBasedRecommender(**fit_dict_ibr)
ibr.fit(tr_info, train, tgt_tracks)
notipy.notify('IBR fitted!')

  1%|          | 48/4420 [00:00<00:09, 474.50it/s]

Calculated Indices


  0%|          | 40/100000 [00:00<04:15, 391.76it/s]

Model URM built
Model URM regularized with IDF!


100%|██████████| 100000/100000 [13:29<00:00, 123.49it/s]


Similarity built


In [92]:
alpha = 0.6
S_ensemble = rs.merge_similarities(tsr.S, ibr.S, alpha)

In [93]:
recommendetions = rs.generic_similarity_based_recommend(S_ensemble, tr_info, tgt_tracks, tgt_playlists, train, secondary_sorting=False)
notipy.notify('Recommending completed!')

  1%|          | 25/4420 [00:00<00:18, 242.07it/s]

URM built


100%|██████████| 4420/4420 [00:19<00:00, 227.56it/s]


In [94]:
map_eval = rs.evaluate(recommendetions, test, 'MAP')
notipy.notify('Evaluation completed!')

In [95]:
run_data = {'recommender_1' : tsr.__class__.__name__,
            'recommender_2' : ibr.__class__.__name__,
            'fit_parameters_1' : fit_dict_tsr,
            'fit_parameters_2' : fit_dict_ibr,
            'alpha_ensemble' : alpha,
            'evaluation_result' : map_eval}

In [96]:
with open('runs_data.json', 'a') as fp:
    json.dump(run_data, fp, indent=2)
    fp.write('\n')

In [97]:
map_eval

0.08471568627450975